In [1]:
# Instalaciones e importaciones
!pip install roboflow --quiet
!pip install ultralytics==8.0.20

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0

In [2]:
from roboflow import Roboflow
import yaml
import os
import shutil
import requests
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.3/78.2 GB disk)


In [3]:
HOME = os.getcwd()
print(HOME)

/content


In [4]:
!mkdir {HOME}/dataset
%cd {HOME}/dataset

/content/dataset


In [5]:
MY_API_KEY = "DU6ZL8wAWd7VgPyvebiU"

# Inicialización y descarga del proyecto y dataset específicos desde Roboflow
rf = Roboflow(api_key=MY_API_KEY)
project = rf.workspace("disco").project("do2")
dataset = project.version(1).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.134 is required but found version=8.0.20, to fix: `pip install ultralytics==8.0.134`



Extracting Dataset Version Zip to DO2-1 in yolov8:: 100%|██████████| 12820/12820 [00:01<00:00, 9828.90it/s] 


In [6]:
# Modifica el archivo data.yaml
data_yaml_path = os.path.join(dataset.location, 'data.yaml')
with open(data_yaml_path, 'r') as file:
    data = yaml.safe_load(file)

data['train'] = '../DO2-1/train/images'
data['val'] = '../DO2-1/valid/images'
data['test'] = '../DO2-1/test/images'

with open(data_yaml_path, 'w') as file:
    yaml.safe_dump(data, file)

%cd {HOME}

/content


In [7]:
# Combinaciones de hiperparametros seleccionados
weight = 0.0003
epochs = 5
batch = 32
nbatch = 64
Optimizador = "SGD"
mom = 0.937

# Entrena el modelo YOLO
!yolo task=detect mode=train model=yolov8s.pt data='{dataset.location}/data.yaml' imgsz=640 epochs={epochs} batch={batch} nbs={nbatch} optimizer={Optimizador} momentum={mom} weight_decay={weight} name=None seed=0 patience=50 close_mosaic=10 resume=False overlap_mask=True mask_ratio=4 dropout=0.0 val=True save_json=True conf=None iou=0.7 max_det=300 plots=True show=True save_txt=True save_conf=True save_crop=True retina_masks=True boxes=True lr0=0.01 lrf=0.01 warmup_epochs=3.0 warmup_momentum=0.8 warmup_bias_lr=0.1 box=10.0 cls=0.5 dfl=1.5 label_smoothing=0.1 hsv_h=0.0 hsv_s=0.0 hsv_v=0.0 degrees=5.0 translate=0.1 scale=0.3 shear=0.0 perspective=0.001 flipud=0.0 fliplr=0.5 mosaic=1.0 mixup=0.0 copy_paste=0.0

100% 21.5M/21.5M [00:00<00:00, 243MB/s]

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=/content/dataset/DO2-1/data.yaml, epochs=5, patience=50, batch=32, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, save_json=True, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=ultralytics/assets/, show=True, save_txt=True, save_conf=True, save_crop=True, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=True, boxes=True, format=torchscript, keras=False, optimize=False, int8=Fal

In [ ]:
# Validar el modelo entrenado .pt
!yolo task=detect mode=val model='{save_directory}/train/weights/best.pt' data={dataset.location}/data.yaml imgsz=640 batch={batch} save_json=True save_hybrid=False conf=0.001 iou=0.6 max_det=300 plots=True


In [ ]:
version = project.version(1)
version.deploy("yolov8", "/content/yolov8s.pt")